In [ ]:
#import modules
import pandas as pd
import glob

In [ ]:
# path of the folder
path = r'F:\sReport'

# reading all the excel files
filenames = glob.glob(path + "\*.xlsx")
print('File names:', filenames)
  
# initializing empty data frame
finalexcelsheet = pd.DataFrame()

In [ ]:
  
# to iterate excel file one by one 
# inside the folder
for file in filenames:
  
    # combining multiple excel worksheets
    # into single data frames
    df = pd.concat(pd.read_excel(
      file, sheet_name=None), ignore_index=True, sort=False)
  
    # appending excel files one by one
    finalexcelsheet = finalexcelsheet.append(
      df, ignore_index=True)

In [ ]:
# to print the combined data
print('Final Sheet:')
display(finalexcelsheet)

In [ ]:
finalexcelsheet.info()

In [ ]:
#Data PreProcessing
#Format Data to import in SQL Successfully
#all my SQL columns are nVarchar
finalexcelsheet['X'] = [format(i, '.3f') for i in finalexcelsheet['X']]
finalexcelsheet['Y'] = [format(i, '.3f') for i in finalexcelsheet['Y']]
finalexcelsheet['Z'] = [format(i, '.3f') for i in finalexcelsheet['Z']]


In [ ]:
finalexcelsheet['C'].fillna('Null')

In [ ]:
finalexcelsheet.info()

In [ ]:
import pyodbc

In [ ]:
# Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=ServerName;'
                      'Database=DatabaseName;'
                      'Trusted_Connection=yes;'
                      'UID=sa;'
                      'PWD=sa;')
cursor = conn.cursor()
cursor.fast_executemany = True

In [ ]:
#To remove data from destination and insert again
cursor.execute('''delete TableName''')
conn.commit()

In [ ]:
#insertt into SQL server Method 1
for row in finalexcelsheet.itertuples():
    cursor.execute('''
                INSERT INTO Sale ([A],[B],[C])
                VALUES (?,?,?)
                ''',
                row.X, 
                row.Y,
                row.Z
                )
conn.commit()
cursor.close()
conn.close()

In [ ]:
#insertt into SQL server Method 2
# PREPARED STATEMENT (NO DATA)
query = '''INSERT INTO Sale ([A],[B],[C])
                VALUES (?,?,?)'''

# LIST OF TUPLES FOR PARAMS
data = [(row.X, 
         row.Y,
        row.Z,
) for row in finalexcelsheet.itertuples()]

# EXECUTE STATEMENT AND BIND PARAMS
cursor.executemany(query, data)
cursor.commit()
cursor.close()
conn.close()